<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/WS%202020%20Course%20Notes/Session%206.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO

# **Data Science for Geoscientists - Winter Semester 2020**
# **Session 6 - Regressions - 11th November 2020**

Multivariate stats, regressions, curve fitting
Introduce main project

# Part 6.1 - Ice Breaker - *Discussion*

# Part 6.2 - Introduction to Regressions - *Mini-lecture*

# Part 6.3 - Linear Regression Excercise - *Workshop*



# Part 6.4 - Data Description Excercise - *Workshop*

# Part 6.5 - Robustness of Statistics - *Mini-lecture*

# Part 6.6 - Time Series Comparisons - *Mini-lecture*

# Part 6.7 - Time Series Comparisons Excercise - *Workshop*

# Part 6.8 - Open Time Series Workshop - *Workshop*

# Part 6.9 - Assignment